In [ ]:
import pandas as pd
from google.colab import drive
from pathlib import Path


In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = Path("/content/drive/MyDrive/Colab Notebooks/filtered_news_dataset_single_company.csv")

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

,pubtime,language,char_count,medium_name,head_clean,content_clean,source_file,company,year,month,year_month,text_norm,company_norm
0,2024-02-23 11:49:11+00:00,de,4232.0,cash.ch,Rosengren nimmt bei ABB den Hut und übergibt a...,Chefwechsel bei ABB: Nach vier Jahren an der S...,abb.csv,ABB,2024,2,2024-02,rosengren nimmt bei abb den hut und ubergibt a...,abb
1,2022-09-30 06:37:00+00:00,de,1532.0,cash.ch,ABB trennt sich von restlichem Stromnetz-Geschäft,Der Elektrotechnikkonzern ABB verkauft die ver...,abb.csv,ABB,2022,9,2022-09,abb trennt sich von restlichem stromnetz-gesch...,abb
2,2023-05-19 22:00:00+00:00,de,4457.0,Aargauer Zeitung / MLZ,Fitness-Pionier übergibt an Sohn,Im April 1989 eröffnete Armin Vock das Fitness...,abb.csv,ABB,2023,5,2023-05,fitness-pionier ubergibt an sohn im april 1989...,abb
3,2024-03-27 23:00:00+00:00,de,2984.0,Aargauer Zeitung / MLZ,Eine Kita in der eigenen Firma,Die Dottiker Schäfer Holzbautechnik gründet ei...,abb.csv,ABB,2024,3,2024-03,eine kita in der eigenen firma die dottiker sc...,abb
4,2021-07-02 03:30:00+00:00,de,8120.0,nzz.ch,Das neue Highlight in der Startup-Szene: Die S...,Die Software von drei Münchner Gründern durchl...,abb.csv,ABB,2021,7,2021-07,das neue highlight in der startup-szene: die s...,abb


In [ ]:
assert {"company", "head_clean", "content_clean", "pubtime"}.issubset(df.columns), \
    "Expected columns company/head_clean/content_clean/pubtime are missing."

# Ensure 'pubtime' is converted to datetime, handle errors gracefully
df["pubtime"] = pd.to_datetime(df["pubtime"], errors="coerce")

# Combine headline + a short snippet of body text
SNIPPET_LEN = 500  # Set as reasonable default if not set elsewhere

def build_text(row):
    head = str(row.get("head_clean", "")).strip()
    body = str(row.get("content_clean", "")).strip()
    return (head + " " + body[:SNIPPET_LEN]).strip()

tmp = pd.DataFrame({
    "date":     df["pubtime"].dt.date,  # Will be NaT for unparsable dates
    "ticker":   df["company"].astype(str),
    "language": df["language"].astype(str),
    "text":     df.apply(build_text, axis=1).astype(str)
})

# Drop rows with missing date, ticker, or text, or very short texts
tmp = tmp.dropna(subset=["date", "ticker", "text"])
tmp = tmp[tmp["text"].str.len() > 3].copy()

print("Rows after cleaning:", len(tmp))
tmp.head(3)



Rows after cleaning: 135435


,date,ticker,language,text
0,2024-02-23,ABB,de,Rosengren nimmt bei ABB den Hut und übergibt a...
1,2022-09-30,ABB,de,ABB trennt sich von restlichem Stromnetz-Gesch...
2,2023-05-19,ABB,de,Fitness-Pionier übergibt an Sohn Im April 1989...


In [ ]:
# drop unparsable dates if any slipped through
tmp = tmp.dropna(subset=["date"])

# normalize ticker format
tmp["ticker"] = tmp["ticker"].str.strip().str.upper()


In [ ]:
print("Rows after cleaning:", len(tmp))

Rows after cleaning: 135435


In [ ]:
# --- Chunk 3: language distribution & strategy ---
lang_counts = tmp["language"].str.lower().value_counts()
display(lang_counts.head(10))

# we know your set is mixed; force multilingual to be explicit
MODEL_STRATEGY = "multilingual"
print("Model strategy:", MODEL_STRATEGY)


,count
language,
de,110640
fr,23129
it,1109
en,557


Model strategy: multilingual


## XLM Roberta

In [ ]:
# --- Chunk 4: load multilingual model + define scorer ---

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"  # multilingual

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
mdl = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device).eval()

def score_texts(texts, max_len=256, batch_size=64):
    """
    texts: list[str]
    returns: np.ndarray of shape (len(texts),) with scores in [-1, 1]
             score = P(positive) - P(negative)
    """
    out = np.zeros(len(texts), dtype="float32")
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            enc = tok(
                batch,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=max_len
            ).to(device)
            logits = mdl(**enc).logits.detach().cpu().numpy()
            probs = softmax(logits, axis=1)   # order: [neg, neu, pos]
            out[i:i+batch_size] = probs[:,2] - probs[:,0]
    return out




In [ ]:
import math
# --- Chunk 5: score all articles ---

# optional: progress bar (comment out if you don't have tqdm)
try:
    from tqdm.auto import tqdm
    use_tqdm = True
except Exception:
    use_tqdm = False

texts = tmp["text"].tolist()
N = len(texts)
scores = np.zeros(N, dtype="float32")

batch = 64  # you can raise to 128 if you have plenty of GPU RAM
max_len = 256

mdl.eval()
with torch.no_grad():
    it = range(0, N, batch)
    if use_tqdm: it = tqdm(it, total=math.ceil(N/batch), desc="Scoring")
    for i in it:
        batch_texts = texts[i:i+batch]
        enc = tok(
            batch_texts,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=max_len
        ).to(device)
        logits = mdl(**enc).logits.detach().cpu().numpy()
        probs = softmax(logits, axis=1)   # [neg, neu, pos]
        scores[i:i+batch] = probs[:, 2] - probs[:, 0]  # pos - neg in [-1, 1]

tmp["sent_score"] = scores

# quick sanity check
tmp[["date","ticker","language","sent_score"]].head(5)
tmp["sent_score"].describe()


Scoring:   0%|          | 0/2117 [00:00<?, ?it/s]

,sent_score
count,135435.000000
mean,-0.209697
std,0.275455
min,-0.949389
25%,-0.363764
50%,-0.169387
75%,-0.047132
max,0.917039


## German FinBERT

In [ ]:
# --- Chunk 5B: Add German FinBERT sentiment ---

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
FINBERT_DE_MODEL = "oliverguhr/german-sentiment-bert"

print("Loading German FinBERT model…")
tok_de = AutoTokenizer.from_pretrained(FINBERT_DE_MODEL, use_fast=False)
mdl_de = AutoModelForSequenceClassification.from_pretrained(FINBERT_DE_MODEL).to(device).eval()

def score_texts_de(texts, max_len=256, batch_size=64):
    """German FinBERT scoring"""
    out = np.zeros(len(texts), dtype="float32")
    mdl_de.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            enc = tok_de(
                texts[i:i+batch_size],
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=max_len
            ).to(device)
            logits = mdl_de(**enc).logits.detach().cpu().numpy()
            probs = softmax(logits, axis=1)   # [neg, neu, pos]
            out[i:i+batch_size] = probs[:, 2] - probs[:, 0]
    return out

# score only German rows
mask_de = tmp["language"].str.lower().eq("de")
texts_de = tmp.loc[mask_de, "text"].tolist()
print(f"Scoring {len(texts_de):,} German articles with FinBERT…")

scores_de = score_texts_de(texts_de)
tmp.loc[mask_de, "sent_score_finbert"] = scores_de

# other languages: keep NaN (no FinBERT)
tmp["sent_score_finbert"] = tmp["sent_score_finbert"].fillna(np.nan)

# rename old XLM-R column for clarity
tmp = tmp.rename(columns={"sent_score": "sent_score_xlm"})

print("Done. New columns:")
print(tmp[["language", "sent_score_xlm", "sent_score_finbert"]].head(5))


Loading German FinBERT model…


tokenizer_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Scoring 110,640 German articles with FinBERT…
Done. New columns:
  language  sent_score_xlm  sent_score_finbert
0       de        0.010688            0.999740
1       de       -0.053976            0.999977
2       de       -0.391919            0.008612
3       de       -0.038528            0.999985
4       de        0.047593            0.999917


In [ ]:
tmp.head(20)

,date,ticker,language,text,sent_score_xlm,sent_score_finbert
0,2024-02-23,ABB,de,Rosengren nimmt bei ABB den Hut und übergibt a...,0.010688,0.999740
1,2022-09-30,ABB,de,ABB trennt sich von restlichem Stromnetz-Gesch...,-0.053976,0.999977
2,2023-05-19,ABB,de,Fitness-Pionier übergibt an Sohn Im April 1989...,-0.391919,0.008612
3,2024-03-27,ABB,de,Eine Kita in der eigenen Firma Die Dottiker Sc...,-0.038528,0.999985
4,2021-07-02,ABB,de,Das neue Highlight in der Startup-Szene: Die S...,0.047593,0.999917
5,2019-03-06,ABB,fr,Un pionnier de l’hydrogène débarque au Salon d...,0.323068,NaN
6,2018-02-10,ABB,fr,Les investisseurs s’interrogent sur la santé f...,0.300536,NaN
7,2024-11-04,ABB,de,"Eine 100-jährige Erfindung, wichtiger als die ...",-0.185489,0.991981
8,2019-02-03,ABB,de,ABB-CEO: «Europa kann bei Robotern eine sehr s...,0.165671,0.977820
9,2021-10-27,ABB,de,ABB erhält Grossauftrag der Deutschen Bahn für...,0.005857,0.999985


In [ ]:
df['content_clean'][16]

'Der Technologiekonzern ABB strebt für die Division Turbocharging, welche seit einigen Wochen Acelleron heisst, weiterhin eine Abspaltung an.\n\n SDA \n\nDer Entscheid soll bis zum Ende des zweiten Quartals 2022 gefällt werden, wie CEO Björn Rosenberg gemäss den Redeunterlagen anlässlich der virtuell abgehaltenen Generalversammlung sagte.\n\n"Wir beabsichtigen zwar nach wie vor uns vom Turboladergeschäft - Accelleron - zu trennen, aber wir werden den endgültigen Entscheid nicht überstürzen", sagte er mit Blick auf die Turbulenzen an den Aktienmärkten wegen des Kriegs in der Ukraine. Unverändert präferiert er eine Ausgliederung von Accelleron an der Schweizer Börse.\n\nDazu müssten aber natürlich die Aktionäre zustimmen. Rosengren hatte bereits im Februar durchblicken lassen, dass eine Abgabe der Division an die Aktionäre über ein Spin Off die wahrscheinlichste Variante ist.\n\nDie Division Turbocharging, welche das Geschäft mit Turboladern für Diesel- und Gasmotoren umfasst, wurde erst

In [ ]:
output_path = Path("/content/drive/MyDrive/Colab Notebooks/scored_news_data.csv")
tmp.to_csv(output_path, index=False)
print(f"Saved scored data to {output_path}")

Saved scored data to /content/drive/MyDrive/Colab Notebooks/scored_news_data.csv


In [ ]:
path_score = Path("/content/drive/MyDrive/Colab Notebooks/scored_news_data.csv")

In [ ]:
tmp = pd.read_csv(path_score)

In [ ]:
tmp.head(20)

,date,ticker,language,text,sent_score_xlm,sent_score_finbert
0,2024-02-23,ABB,de,Rosengren nimmt bei ABB den Hut und übergibt a...,0.010688,0.999740
1,2022-09-30,ABB,de,ABB trennt sich von restlichem Stromnetz-Gesch...,-0.053976,0.999977
2,2023-05-19,ABB,de,Fitness-Pionier übergibt an Sohn Im April 1989...,-0.391919,0.008612
3,2024-03-27,ABB,de,Eine Kita in der eigenen Firma Die Dottiker Sc...,-0.038528,0.999985
4,2021-07-02,ABB,de,Das neue Highlight in der Startup-Szene: Die S...,0.047593,0.999917
5,2019-03-06,ABB,fr,Un pionnier de l’hydrogène débarque au Salon d...,0.323068,NaN
6,2018-02-10,ABB,fr,Les investisseurs s’interrogent sur la santé f...,0.300536,NaN
7,2024-11-04,ABB,de,"Eine 100-jährige Erfindung, wichtiger als die ...",-0.185489,0.991981
8,2019-02-03,ABB,de,ABB-CEO: «Europa kann bei Robotern eine sehr s...,0.165671,0.977820
9,2021-10-27,ABB,de,ABB erhält Grossauftrag der Deutschen Bahn für...,0.005857,0.999985
